In [2]:
!pip install torch

     |████████████████████████████████| 150.6 MB 186 kB/s  eta 0:00:01


In [6]:
!pip3 install torch_geometric

In [101]:
import torch
from torch_geometric.data import Data
import torch.nn.functional as F

num_classes = 3
edge_index = torch.tensor([[0, 1, 1, 2, 3, 2],
                           [1, 0, 2, 1, 2, 3]], dtype=torch.long)
x = torch.tensor([[], [], [], []], dtype=torch.float)
y = torch.tensor([[0, 1, 0], [0, 0, 1], [1, 0, 1], [1, 0, 0]], dtype=torch.float)
train_mask = torch.tensor([True, True, False, False], dtype=torch.bool)
test_mask = torch.tensor([False, False, True, True], dtype=torch.bool)

data = Data(x=x, edge_index=edge_index, y=y, train_mask=train_mask, test_mask=test_mask)

In [102]:
class GCNModel(torch.nn.Module):
    def __init__(self, num_nodes, hidden_size=32):
        super(GCNModel, self).__init__()
        self.node_embedding = torch.nn.Embedding(num_nodes, hidden_size)
        # Initialize the embeddings with small random values
        torch.nn.init.normal_(self.node_embedding.weight, std=0.1)
        self.conv1 = GCNConv(hidden_size, 16)
        self.conv2 = GCNConv(16, num_classes)

    def forward(self, data):
        edge_index = data.edge_index
        x = self.node_embedding.weight

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return F.softmax(x, dim=1)

In [103]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = GCNModel(len(data.x)).to(device)
data = data.to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01, weight_decay=5e-4)

model.train()
for epoch in range(100):
    optimizer.zero_grad()
    out = model(data)
    loss = F.cross_entropy(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    if(epoch % 10 == 0):
        print(epoch, loss)
    optimizer.step()


0 tensor(1.0996, grad_fn=<DivBackward1>)
10 tensor(1.0188, grad_fn=<DivBackward1>)
20 tensor(1.0038, grad_fn=<DivBackward1>)
30 tensor(0.9899, grad_fn=<DivBackward1>)
40 tensor(1.0105, grad_fn=<DivBackward1>)
50 tensor(0.6312, grad_fn=<DivBackward1>)
60 tensor(0.6712, grad_fn=<DivBackward1>)
70 tensor(0.7350, grad_fn=<DivBackward1>)
80 tensor(0.5685, grad_fn=<DivBackward1>)
90 tensor(0.5911, grad_fn=<DivBackward1>)


In [107]:
model.eval()
pred = model(data)
# Convert probabilities to binary predictions
binary_pred = (pred >= 1.0/num_classes).float()
# Check if each prediction matches the label for each instance
correct = (binary_pred[data.test_mask] == data.y[data.test_mask]).sum() / num_classes
acc = int(correct) / int(data.test_mask.sum())
print(f'Accuracy: {acc:.4f}')

tensor([[5.2608e-10, 2.0960e-11, 1.0000e+00],
        [3.0928e-11, 2.9595e-14, 1.0000e+00]], grad_fn=<IndexBackward0>)
tensor([[0., 0., 1.],
        [0., 0., 1.]]) tensor([[1., 0., 1.],
        [1., 0., 0.]])
tensor([[False,  True,  True],
        [False,  True, False]])
tensor(1.)
Accuracy: 0.5000
